# Setup

In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
from main import *
import math

In [3]:
set_seed(config["seed"])

In [4]:
# load test dataset
transform = get_transform()
test_dataset = MNIST(
    root="data/mnist/test", train=False, download=True, transform=transform
)

# data loader
test_loader = DataLoader(
    test_dataset,
    batch_size=config["batch_size"],
    num_workers=config["num_workers"],
    shuffle=False,
    pin_memory=True,
    drop_last=False,
)

# load model
model = CNN()
model.load_state_dict(torch.load(config["model_path"]))
model.to(device)

CNN(
  (conv1): Conv2d(1, 20, kernel_size=(5, 5), stride=(1, 1))
  (pool1): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (relu1): ReLU()
  (conv2): Conv2d(20, 40, kernel_size=(5, 5), stride=(1, 1))
  (pool2): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (relu2): ReLU()
  (dropout): Dropout2d(p=0.25, inplace=False)
  (fc1): Linear(in_features=640, out_features=400, bias=True)
  (relu3): ReLU()
  (fc2): Linear(in_features=400, out_features=10, bias=True)
)

# Save Activations

In [46]:
class ActivationsHook:
    def __init__(self, model, modules):
        self.model = model
        self.modules = modules
        self.activations = {}
        self.weights = {}
        self.handles = []

    def register_hooks(self):
        for module in self.modules:
            print("registered hook for", module)
            self.handles.append(module.register_forward_hook(self))
            self.activations[module] = []

    def unregister_hooks(self):
        for handle in self.handles:
            handle.remove()
        self.handles = []
            
    def save_activations(self, fn, *args, **kwargs):
        self.activations = {}
        self.weights = {}
        self.handles = []
        
        # register hooks
        self.register_hooks()
        
        # run the model
        outputs = fn(*args, **kwargs)
        
        # get vector of each neuron
        for module, tensors in self.activations.items():            
            # stack all the activations to have shape (N, ...), where N = num examples
            tensors = torch.cat(tensors)
            self.activations[module] = tensors
        
        # unregister hooks
        self.unregister_hooks()
        return outputs
        
    def __call__(self, module, module_in, module_out):
        self.activations[module].append(module_out)
        self.weights[module] = module_in

In [47]:
try:
    print('reset')
    pruner.reset()
except:
    pass

modules_to_register = [module for module in model.modules() if isinstance(module, nn.ReLU)]

act_hook = ActivationsHook(model, modules_to_register)

reset


In [48]:
criterion = nn.CrossEntropyLoss()
test_loss, test_acc = act_hook.save_activations(eval_fn, model, test_loader, criterion)
print(f"Test Accuracy: {test_acc:.3f}")
print(f"Test Loss: {test_loss:.3f}")

registered hook for ReLU()
registered hook for ReLU()
registered hook for ReLU()
Test Accuracy: 0.992
Test Loss: 0.070


In [49]:
print(len(act_hook.activations))

3


In [55]:
for module, activations in act_hook.activations.items():
    print(module, tuple(activations.shape), activations.flatten().detach().cpu().numpy()[:5], sep="\n", end="\n\n")

ReLU()
(10000, 20, 12, 12)
[0.12973969 0.12973969 0.12973969 0.12973969 0.12973969]

ReLU()
(10000, 40, 4, 4)
[0.00431408 1.1014757  3.9778383  3.0417483  0.        ]

ReLU()
(10000, 400)
[0.        0.        0.        2.9065669 0.       ]



# Vector Similarity

In [50]:
def cosine_angle(a, b):
    inner_product = torch.dot(a.flatten(), b.flatten())
    a_norm = torch.norm(a)
    b_norm = torch.norm(b)
    return torch.acos(inner_product / (a_norm * b_norm))

In [54]:
a = torch.ones((10))
b = torch.clone(a)
cosine_angle(a, b)

tensor(0.)

In [55]:
a = torch.ones((10))
b = -a
cosine_angle(a, b)

tensor(3.1416)

In [56]:
a = torch.tensor([[11., 21., 10.]])
b = torch.tensor([[12., -2., 14.]])
cosine_angle(a, b)

tensor(1.0679)

In [60]:
a = torch.rand((10, 5))
b = torch.rand((10, 5))
cosine_angle(a, b)

tensor(0.7268)

# Neuron Vectors

In [57]:
for module, activations in act_hook.activations.items():
    # permute the activations such that each neuron is described
    # as a vector of length N (ie last dim of tensor is N, the number of examples)
    dims = list(range(1, len(activations.shape)))
    dims.append(0)
    activations = activations.permute(dims)
    
    print(module, tuple(activations.shape), activations.flatten().detach().cpu().numpy()[:5], sep="\n", end="\n\n")

ReLU()
(20, 12, 12, 10000)
[0.12973969 0.12973969 0.12973969 0.12973969 0.12973969]

ReLU()
(40, 4, 4, 10000)
[0.00431408 0.         0.         0.         0.0743544 ]

ReLU()
(400, 10000)
[0. 0. 0. 0. 0.]

